<a href="https://colab.research.google.com/github/TIN310/TTNT/blob/main/TTNT_Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Thuật giải Minimax **

khai báo các biến
X và O: Ký hiệu 2 người chơi trên bàn cờ.

EMPTY: Đại diện cho ô trống trên bàn cờ (chưa có X hoặc O).

user: Biến lưu ký hiệu người chơi (X hoặc O).

ai: Biến lưu ký hiệu AI (X hoặc O), luôn ngược lại với người chơi.

board_size: Kích thước của bàn cờ NxN. Mặc định là 3, nghĩa là bàn cờ 3x3.

Trong chương trình, biến này có thể thay đổi nếu người chơi muốn chơi 4x4, 5x5…

In [89]:
import copy
import math
import random
import numpy
X = "X"
O = "O"
EMPTY = None
user = None
ai = None
board_size = 3


khởi tạo 1 àn cờ NxN trống với mỗi EMPTY là 1 ô trống trên bàn cờ

In [90]:
def initial_state(n):
    """Trả về bàn cờ NxN trống"""
    return [[EMPTY for _ in range(n)] for _ in range(n)]

Xác định lượt đi của người chơi (ai hoặc user )dựa trên trạng thái hiện tại của board

VD :
nếu có 0 nước đi, lượt của AI (giả định AI là người đi trước). Nếu có 1 nước đi, lượt của người dùng. Nếu có 2 nước đi, lượt của AI, v.v.

In [91]:
def player(board):
    count = 0
    for row in board:
        for cell in row:
            if cell:
                count += 1
    return ai if count % 2 == 0 else user

xác định tất cả các bước đi hợp lệ (các ô trống ) , nếu ô đó = EMPTY (ô trống ) thì ô đó sẽ được thêm vào tập hợp res , cuối cùng hàm sẽ trả về tập hợp res qua return res nơi chứa tất cả các tọa độ  của các ô trống trên bàn cờ

In [92]:
def actions(board):
  res = set()
  board_len = len(board)
  for i in range(board_len):
    for j in range(board_len):
      if board[i][j] == EMPTY:
        res.add((i, j))
  return res

 Mô phỏng kết quả của việc thực hiện một hành động (nước đi) cụ thể trên một bàn cờ.
 Gọi hàm player(board) để xác định người chơi hiện tại đang có lượt đi. Đây có thể là ai hoặc user.
 result_board = copy.deepcopy(board): Tạo một bản sao sâu (deep copy) của bàn cờ hiện tại. Để đảm bảo bàn cờ gốc không bị thay đổi , điều này cho phép chúng ta thử ngiệm các nước đi mà không bị ảnh hưởng đến trạng thái ban đầu của trò chơi

In [93]:
def result(board, action):
    curr_player = player(board)
    new_board = copy.deepcopy(board)
    i, j = action
    new_board[i][j] = curr_player
    return new_board

get_horizontal_winner(board) này được tạo ra để kiểm tra xem có bất kỳ hàng ngang nào trên bàn cờ có người thắng cuộc hay không.
1.   Khởi tạo một biến winner_val với giá trị None
dùng để lưu trữ ký hiệu của người thắng nếu thấy và vẫn giữ None nếu không có ai thắng
2.   winner_val = board[i][0] : hàm đang giả định rằng nếu có người thắng ở  hàng này thì sẽ là người đánh đầu tiên .



In [94]:
def get_horizontal_winner(board):
    n = len(board)
    for i in range(n):
        potential_winner = board[i][0]
        if potential_winner is None:
            continue
        is_winner = all(board[i][j] == potential_winner for j in range(n))
        if is_winner:
            return potential_winner
    return None

Hàm get_vertical_winner(board) này được tạo ra để kiểm tra xem có người thắng cuộc theo cột dọc trên bàn cờ hay không.


1.  winner_val = None: Biến này được khởi tạo để lưu trữ ký hiệu của người chơi thắng (X hoặc O) nếu tìm thấy một cột thắng.
2.   winner_val = board[0][i]: Khi bắt đầu kiểm tra một cột mới, hàm gán giá trị của ô đầu tiên trong cột đó (board[0][i]) cho winner_val. Đây là giả định rằng nếu có người thắng ở cột này, người đó phải là người đã điền vào ô đầu tiên.
3.   if board[j][i] != winner_val:: Trong vòng lặp này, giá trị của ô hiện tại (board[j][i]) được so sánh với winner_val (là giá trị của ô đầu tiên trong cột).
*Lưu ý :Nếu ô hiện tại khác với winner_val, điều đó có nghĩa là cột này không thể có người thắng với winner_val là biểu tượng của họ. Vì vậy, winner_val được đặt lại thành None.*




In [95]:
def get_vertical_winner(board):
    n = len(board)
    for j in range(n):
        potential_winner = board[0][j]
        if potential_winner is None:
            continue
        is_winner = all(board[i][j] == potential_winner for i in range(n))
        if is_winner:
            return potential_winner
    return None

get_diagonal_winner(board) này là để kiểm tra xem có người chơi nào đã thắng cuộc bằng cách tạo thành một đường thẳng trên một trong hai đường chéo của bàn cờ hay không.
1. winner_val = board[0][0]: Gán giá trị của ô đầu tiên trên đường chéo chính (ô (0,0)) cho winner_val. Điều này giả định rằng nếu có người thắng trên đường chéo này, thì người đó phải là người đã điền vào ô (0,0).
2. for i in range(board_len): Duyệt qua các ô trên đường chéo chính (các ô có tọa độ (i, i)).
3.   if board[i][i] != winner_val: winner_val = None: Nếu giá trị của ô hiện tại trên đường chéo không khớp với winner_val đã được thiết lập (từ ô (0,0)), thì đặt winner_val về None. Điều này có nghĩa là đường chéo này không có người thắng với ký hiệu ban đầu.





In [96]:
def get_diagonal_winner(board):
    winner_val = None
    board_len = len(board)
    potential_winner_main = None
    is_main_diagonal_winner = True
    for i in range(board_len):
        if potential_winner_main is None and board[i][i] != EMPTY:
            potential_winner_main = board[i][i]
        elif board[i][i] != potential_winner_main or board[i][i] == EMPTY:
            is_main_diagonal_winner = False
            break

    if is_main_diagonal_winner and potential_winner_main is not None:
        return potential_winner_main
    potential_winner_anti = None
    is_anti_diagonal_winner = True
    for i in range(board_len):
        j = board_len - 1 - i
        if potential_winner_anti is None and board[i][j] != EMPTY:
            potential_winner_anti = board[i][j]
        elif board[i][j] != potential_winner_anti or board[i][j] == EMPTY:
            is_anti_diagonal_winner = False
            break

    if is_anti_diagonal_winner and potential_winner_anti is not None:
        return potential_winner_anti

    return winner_val

Hàm winner(board) có ý nghĩa là kiểm tra toàn diện xem có người thắng cuộc trên bàn cờ hay chưa bằng cách kết hợp kết quả từ các hàm kiểm tra thắng theo hàng ngang, hàng dọc và đường chéo.

In [97]:
def winner(board):
    return get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board)


terminal(board): dùng để xác định xem ván cờ đã kết thúc hay chưa
1. if winner(board) != None:
Kiểm tra xem có người chơi nào đã thắng hay chưa.
2. return winner(board)
Nếu đã có người thắng thì sẽ trả về chính người thắng đó (ví dụ "X" hoặc "O").
3.  if j == EMPTY:
Kiểm tra xem ô hiện tại có phải là ô trống không.
*Nếu còn ít nhất một ô trống, nghĩa là trò chơi vẫn có thể tiếp tục.*




In [98]:
def terminal(board):
    if winner(board) is not None:
        return True
    return all(cell is not EMPTY for row in board for cell in row)


utility(board):trả về giá trị đánh giá cuối cùng của ván cờ theo quy ước Minimax
1. winner_val = winner(board)
Gọi hàm winner(board) để kiểm tra xem có người chơi nào thắng ván cờ hay không.
2.  if winner_val == X:
Kiểm tra xem người thắng có phải là người chơi X hay không.
1.  return 1:
Nếu X thắng, hàm trả về
2.  elif winner_val == O:
Nếu X không thắng, kiểm tra tiếp xem người thắng có phải O hay không.





In [99]:
def utility (board):
  winner_val = winner(board)
  if winner_val == X:
    return 1
  elif winner_val == O :
    return -1
  return 0

Hàm minimax(board):
Hàm này là trung tâm của thuật toán Minimax. Nó tìm kiếm nước đi tối ưu nhất cho người chơi hiện tại trên bàn cờ, giả định rằng cả người chơi hiện tại và đối thủ đều sẽ chơi một cách tối ưu để đạt được kết quả tốt nhất cho riêng họ.

**Các bước hoạt động:**

current_player = player(board):

Đầu tiên, hàm xác định người chơi hiện tại đang có lượt đi bằng cách gọi hàm player(board).
Xác định lượt của người chơi MAX (thường là X):

if current_player == X:
Nếu người chơi hiện tại là X (người chơi MAX), mục tiêu của họ là tối đa hóa điểm số.
max_val = -math.inf: Khởi tạo max_val (giá trị tối đa tìm được) bằng âm vô cực. Mọi giá trị hợp lệ sẽ lớn hơn giá trị này.

best_move = None: Khởi tạo best_move (nước đi tốt nhất) là None.

for action in actions(board):
Hàm duyệt qua tất cả các nước đi hợp lệ (action) mà người chơi X có thể thực hiện trên bàn cờ hiện tại.

value = minValue(result(board, action)):
Đây là phần cốt lõi: Với mỗi nước đi tiềm năng của X, hàm sẽ tạo ra một trạng thái bàn cờ mới (result(board, action)).
Sau nước đi của X, sẽ đến lượt của O (người chơi MIN). Vì vậy, hàm gọi minValue để đánh giá trạng thái mới đó. minValue sẽ tính toán giá trị thấp nhất mà O có thể buộc X phải nhận (tức là O sẽ cố gắng tối thiểu hóa kết quả cho X).

if value > max_val:
Nếu giá trị value trả về từ minValue cao hơn max_val hiện tại, điều này có nghĩa là nước đi action này dẫn đến một kết quả tốt hơn cho X.

max_val = value: Cập nhật max_val bằng value mới.

best_move = action: Cập nhật best_move là action hiện tại.

return best_move: Sau khi xem xét tất cả các nước đi có thể, hàm trả về best_move - nước đi mà X nên thực hiện để đảm bảo kết quả tốt nhất (điểm số cao nhất) cho mình, giả sử O chơi tối ưu.

Xác định lượt của người chơi MIN (thường là O):
else: # current_player == O:
Nếu người chơi hiện tại là O (người chơi MIN), mục tiêu của họ là tối thiểu hóa điểm số của đối thủ (X).

min_val = math.inf: Khởi tạo min_val (giá trị tối thiểu tìm được) bằng dương vô cực. Mọi giá trị hợp lệ sẽ nhỏ hơn giá trị này.

best_move = None: Khởi tạo best_move là None.

for action in actions(board)::
Hàm duyệt qua tất cả các nước đi hợp lệ (action) mà người chơi O có thể thực hiện.

value = maxValue(result(board, action)):
Tương tự, tạo ra trạng thái bàn cờ mới sau nước đi của O.
Sau nước đi của O, sẽ đến lượt của X (người chơi MAX). Vì vậy, hàm gọi maxValue để đánh giá trạng thái mới đó.

maxValue sẽ tính toán giá trị cao nhất mà X có thể đạt được (tức là X sẽ cố gắng tối đa hóa kết quả cho X).

if value < min_val:
Nếu giá trị value trả về từ maxValue thấp hơn min_val hiện tại, điều này có nghĩa là nước đi action này dẫn đến một kết quả tốt hơn cho O (làm giảm điểm số của X).
min_val = value: Cập nhật min_val bằng value mới.

best_move = action: Cập nhật best_move là action hiện tại.

return best_move: Sau khi xem xét tất cả các nước đi có thể, hàm trả về best_move - nước đi mà O nên thực hiện để đảm bảo kết quả tốt nhất (điểm số thấp nhất cho X) cho mình, giả sử X chơi tối ưu.


In [100]:
def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v


def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

Xác định người chơi hiện tại:
current_player = player(board)



Kiểm tra lượt đi hiện tại là X (MAX) hay O (MIN):
Nếu người chơi là MAX (X)

if current_player == X:
    max_val = -math.inf
    best_move = None


max_val = giá trị lớn nhất tìm được, khởi tạo âm vô cực.

best_move = nước đi tốt nhất hiện tại, khởi tạo None.

Duyệt tất cả nước đi hợp lệ:
for action in actions(board):
    value = minValue(result(board, action))


Tạo trạng thái mới sau khi X đi action.

Gọi minValue để xem O (MIN) sẽ phản ứng như thế nào → MIN sẽ cố giảm điểm số của X.

Cập nhật nước đi tốt nhất

if value > max_val:
    max_val = value
    best_move = action


Nếu giá trị mới tốt hơn max_val → cập nhật best_move.

Trả về nước đi tối ưu cho X

return best_move


Nếu người chơi là MIN (O)

else:
    min_val = math.inf
    best_move = None


min_val = giá trị nhỏ nhất tìm được, khởi tạo dương vô cực.

Duyệt tất cả nước đi hợp lệ

for action in actions(board):
    value = maxValue(result(board, action))


Tạo trạng thái mới sau khi O đi action.

Gọi maxValue để xem X (MAX) sẽ phản ứng → X sẽ cố tăng điểm số.

Cập nhật nước đi tốt nhất

if value < min_val:
    min_val = value
    best_move = action


Nếu giá trị mới nhỏ hơn min_val → cập nhật best_move.

Trả về nước đi tối ưu cho O

return best_move


In [101]:

def minimax(board):
    current_player = player(board)
    if current_player == X:
        max_val = -math.inf
        best_move = None
        for action in actions(board):
            value = minValue(result(board, action))
            if value > max_val:
                max_val = value
                best_move = action
        return best_move
    else:
        min_val = math.inf
        best_move = None
        for action in actions(board):
            value = maxValue(result(board, action))
            if value < min_val:
                min_val = value
                best_move = action
        return best_move


In bàn cờ ra

In [102]:
def print_board(board):
    n = len(board)
    for row in board:
        print(" | ".join([cell if cell is not None else " " for cell in row]))
    print("\n")

Chương trình khởi tạo bàn cờ và hỏi người chơi muốn chơi X hay O.

AI sẽ chơi nước đi tối ưu theo Minimax.

Người chơi nhập vị trí ô muốn đánh, chương trình kiểm tra hợp lệ.

Vòng lặp tiếp tục cho đến khi game kết thúc (thắng , thua hoặc hòa).

Bàn cờ được hiển thị sau mỗi nước đi, thông báo kết quả cuối cùng khi game kết thúc.

In [105]:
if __name__ == "__main__":
    # Chọn kích thước bàn cờ NxN
    while True:
        try:
            n = int(input("Nhập kích thước bàn cờ N (ví dụ 3, 4, 5): "))
            if n < 3:
                print("Bàn cờ phải >= 3x3. Nhập lại!")
            else:
                board_size = n
                break
        except ValueError:
            print("Phải nhập số nguyên! Nhập lại!")

    board = initial_state(board_size)

    # Chọn người chơi
    while True:
        user = input("Chọn X hoặc O: ").strip().upper()
        if user not in ["X", "O"]:
            print("Sai ký hiệu! Chỉ được X hoặc O.")
        else:
            break

    ai = "O" if user == "X" else "X"

    print_board(board)

    # Vòng lặp game
    while True:
        current_player = player(board)

        if terminal(board):
            w = winner(board)
            if w:
                print_board(board)
                print(f"Game Over: {w} thắng!")
            else:
                print_board(board)
                print("Game Over: Hòa!")
            break

        if current_player == ai:
            print(f"AI ({ai}) đang nghĩ...")
            move = minimax(board)
            board = result(board, move)
            print("AI đã đi:")
            print_board(board)
        else:
            while True:
                try:
                    row = int(input(f"Hãy nhập hàng (0-{board_size-1}): "))
                    col = int(input(f"Hãy nhập cột (0-{board_size-1}): "))
                    if (0 <= row < board_size) and (0 <= col < board_size):
                        if board[row][col] == EMPTY:
                            board = result(board, (row, col))
                            print_board(board)
                            break
                        else:
                            print("Ô đã có người đi, chọn ô khác!")
                    else:
                        print(f"Hàng và cột phải từ 0 đến {board_size-1}!")
                except ValueError:
                    print("Phải nhập số nguyên! Thử lại.")

Nhập kích thước bàn cờ N (ví dụ 3, 4, 5): 3
Chọn X hoặc O: O
  |   |  
  |   |  
  |   |  


AI (X) đang nghĩ...
AI đã đi:
  | X |  
  |   |  
  |   |  


Hãy nhập hàng (0-2): 0
Hãy nhập cột (0-2): 0
O | X |  
  |   |  
  |   |  


AI (X) đang nghĩ...
AI đã đi:
O | X |  
  | X |  
  |   |  


Hãy nhập hàng (0-2): 1
Hãy nhập cột (0-2): 2
O | X |  
  | X | O
  |   |  


AI (X) đang nghĩ...
AI đã đi:
O | X |  
  | X | O
  | X |  


O | X |  
  | X | O
  | X |  


Game Over: X thắng!
